In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path

from hscpy import parameters
from futils import parse_version

RUNS = 65_001
ABC_4D = True  # infer tau from abc (4d abc)
MU_UNIFORM = False  # uniform or 2**2 steps
PATH2BIN = Path("~").expanduser() / "hsc-results/"
assert PATH2BIN.is_dir()
MY_BIN = "hsc-v4.3.12-x86_64-unknown-linux-gnu"  # TODO add bin used to generate sims
assert MY_BIN
MY_BIN = PATH2BIN / MY_BIN
assert MY_BIN.is_file
MY_BIN

In [ ]:
%%bash -s "$MY_BIN" --out version
"$1" --version

In [ ]:
VERSION = parse_version(version)
PATH2SAVE = Path(f"/data/scratch/hfx923/hsc-draft/{VERSION}")
PATH2SAVE

In [ ]:
# RANGES!!
"""
Up-to-date ranges Feb 13:
    s (i.e. old s/tau) : 0.01 to 0.4
    sigma (i.e. old sigma/tau) : 0.001 to 0.1
    mu: 0.1 to 30
    tau: 0.1 to 10
    N: 25'000 to 900'000
"""
# TRY something new, increase mu space
"""
Up-to-date ranges Feb 13:
    s (i.e. old s/tau) : 0.01 to 0.4
    sigma (i.e. old sigma/tau) : 0.001 to 0.1
    mu: 2, 4, 8, 16, 32, 64, 128, 
    tau: 0.1 to 10
    N: 25'000 to 900'000
"""
cells = np.ones(RUNS, dtype=int) * 100_000
rng = np.random.default_rng(seed=42)

if ABC_4D:
    taus = rng.uniform(0.1, 10, RUNS)
else:
    taus = np.ones(RUNS, dtype=int)
if MU_UNIFORM:
    mu0s = rng.uniform(0.1, 30, RUNS)
else:
    mu0s = rng.choice(np.exp2(np.arange(1, 9)), RUNS)
# eta
etas = rng.uniform(0.001, 0.4, RUNS)
# sigma
sigmas = rng.uniform(0.001, 0.1, RUNS)

assert (etas * taus).max() < 4
assert (sigmas * taus).max() < 1

In [ ]:
if MU_UNIFORM:
    sns.pairplot(
        pd.DataFrame(
            [mu0s, taus, etas, sigmas, cells], index=["mu", "tau", "eta", "sigma", "N"]
        ).T
    )
    plt.show()
else:
    sns.pairplot(
        pd.DataFrame(
            [taus, etas, sigmas, cells], index=["tau", "eta", "sigma", "N"]
        ).T
    )
    plt.show()
    
    plt.hist(mu0s, bins=np.exp2(np.arange(1, 10)), rwidth=0.5, align="left")
    plt.xscale('log', base=2)
    plt.show()
plt.hist(
    parameters.compute_m_background_exp(),
    label="mu background",
    bins=100,
)
plt.legend()
plt.show()

In [ ]:
with open("parameters.txt", "w") as f:
    for tau, mu, mean, std, n in np.column_stack((taus, mu0s, etas, sigmas, cells)):
        c = int(n)
        f.write(
            f"{MY_BIN} -c {c} -y 82 -r 1 --sequential --mean-std {parameters.compute_s_per_division_from_s_per_year(mean, tau)} {parameters.compute_std_per_division_from_std_per_year(std, tau)} --subsamples=216,390,407,380,362,361,367,451,328 --snapshots=0,0,29,38,48,63,76,77,81 {PATH2SAVE} exp-moran --tau-exp {parameters.tau_exp(c)} --mu-exp {mu} --mu-division-exp 1.14 --mu-background-exp {parameters.compute_m_background_exp()} --tau {tau} --mu {mu} --mu-division 1.14 --mu-background {parameters.m_background(tau)}\n"
        )

In [ ]:
%%bash
head parameters.txt

In [ ]:
%%bash --out runs
wc -l parameters.txt

In [ ]:
runs = int(runs.split(" ")[0])
assert RUNS == runs, f"{RUNS} vs {runs}"